# Metody zespołowe

### Cel/Zakres
- Metody zespołowe
  - równoległe
  - sekwencyjne
- Hard / Soft voting
- Bagging
- Boosting

### Przygotowanie danych

In [ ]:
from sklearn import datasets
data_breast_cancer = datasets.load_breast_cancer(as_frame=True);

### Ćwiczenie

In [ ]:
from sklearn.model_selection import train_test_split

# Podział zbioru na testowy i treningowy
X_train, X_test, y_train, y_test = train_test_split(data_breast_cancer['data'][['mean texture', 'mean symmetry']], data_breast_cancer['target'], random_state=42)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

# Trzy modele
tree_clf = DecisionTreeClassifier(random_state=42)
log_clf = LogisticRegression(random_state=42)
knn_clf = KNeighborsClassifier()

# Głosowanie typu "hard"
voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('tc', tree_clf), ('knn', knn_clf)],
    voting='hard'
)
voting_clf.fit(X_train, y_train)

hard_train_acc = accuracy_score(y_train, voting_clf.predict(X_train))
hard_test_acc = accuracy_score(y_test, voting_clf.predict(X_test))
print(f"Dokładność \"Hard Voting\" dla zbioru uczącego: {hard_train_acc}")
print(f"Dokładność\"Hard Voting\" dla zbioru testowego: {hard_test_acc}")

Dokładność "Hard Voting" dla zbioru uczącego: 0.8380281690140845
Dokładność"Hard Voting" dla zbioru testowego: 0.6923076923076923


In [ ]:
# Głosowanie typu "soft"
voting_clf_soft = VotingClassifier(
    estimators=[('lr', log_clf), ('tc', tree_clf), ('knn', knn_clf)],
    voting='soft'
)

voting_clf_soft.fit(X_train, y_train)

soft_train_acc = accuracy_score(y_train, voting_clf_soft.predict(X_train))
soft_test_acc = accuracy_score(y_test, voting_clf_soft.predict(X_test))
print(f"Dokładność \"Soft Voting\" dla zbioru uczącego: {soft_train_acc}")
print(f"Dokładność \"Soft Voting\" dla zbioru testowego: {soft_test_acc}")

Dokładność "Soft Voting" dla zbioru uczącego: 0.9624413145539906
Dokładność "Soft Voting" dla zbioru testowego: 0.6853146853146853


In [ ]:
# obliczenie accuracy dla pojedynczych klasyfikatorów
tree_clf.fit(X_train, y_train)
log_clf.fit(X_train, y_train)
knn_clf.fit(X_train, y_train)

tc_train_acc = accuracy_score(y_train, tree_clf.predict(X_train))
tc_test_acc = accuracy_score(y_test, tree_clf.predict(X_test))

lc_train_acc = accuracy_score(y_train, log_clf.predict(X_train))
lc_test_acc = accuracy_score(y_test, log_clf.predict(X_test))

knn_train_acc = accuracy_score(y_train, knn_clf.predict(X_train))
knn_test_acc = accuracy_score(y_test, knn_clf.predict(X_test))

print(f"Dokładność DRZEWA DECYZYJNEGO dla zbioru uczącego: {tc_train_acc}")
print(f"Dokładność DRZEWA DECYZYJNEGO dla zbioru testowego: {tc_test_acc}")
print(f"Dokładność REGRESJI LOGISTYCZNEJ dla zbioru uczącego: {lc_train_acc}")
print(f"Dokładność REGRESJI LOGISTYCZNEJ dla zbioru testowego: {lc_test_acc}")
print(f"Dokładność KNN dla zbioru uczącego: {knn_train_acc}")
print(f"Dokładność KNN dla zbioru testowego: {knn_test_acc}")

Dokładność DRZEWA DECYZYJNEGO dla zbioru uczącego: 1.0
Dokładność DRZEWA DECYZYJNEGO dla zbioru testowego: 0.6363636363636364
Dokładność REGRESJI LOGISTYCZNEJ dla zbioru uczącego: 0.7230046948356808
Dokładność REGRESJI LOGISTYCZNEJ dla zbioru testowego: 0.7062937062937062
Dokładność KNN dla zbioru uczącego: 0.7723004694835681
Dokładność KNN dla zbioru testowego: 0.6433566433566433


Można zauważyć, że *accuracy* dla Drzewa Decyzyjnego wynosi 1.0 czyli 100%. Wynika to z faktu, iż DecisionTreeClassifier z domyślnymi parametrami nie ma żadnych ograniczeń, jeśli chodzi o wielkość drzewa, czy sposoby podziału. Oznacza to, że drzewo może się idealnie dopasować do zbioru treningowego.

In [ ]:
import pickle
# Lista z accuracy
acc_list = [(tc_train_acc, tc_test_acc),
            (lc_train_acc, lc_test_acc),
            (knn_train_acc, knn_test_acc),
            (hard_train_acc, hard_test_acc),
            (soft_train_acc, soft_test_acc)]
with open("acc_vote.pkl", "wb") as file:
  pickle.dump(acc_list, file)

# Lista z klasyfikatorami
clf_list = [tree_clf, log_clf, knn_clf, voting_clf, voting_clf_soft]
with open("vote.pkl", "wb") as file:
  pickle.dump(clf_list, file)

#### Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier

bag_clf = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=30,
    bootstrap=True,
    random_state=42
)
bag_clf.fit(X_train, y_train)

bag_acc_train = accuracy_score(y_train, bag_clf.predict(X_train))
bag_acc_test = accuracy_score(y_test, bag_clf.predict(X_test))
print(f"Dokładność Bagging dla zbioru treningowego: {bag_acc_train}")
print(f"Dokładność Bagging dla zbioru testowego: {bag_acc_test}")

Dokładność Bagging dla zbioru treningowego: 1.0
Dokładność Bagging dla zbioru testowego: 0.6923076923076923


#### Bagging z wykorzystaniem 50% instancji

In [ ]:
bag_clf_50 = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=30,
    bootstrap=True,
    random_state=42,
    max_samples= 0.5  # maksymalnie 50% instancji do trenowania modelu
)
bag_clf_50.fit(X_train, y_train)

bag_acc_train_50 = accuracy_score(y_train, bag_clf_50.predict(X_train))
bag_acc_test_50 = accuracy_score(y_test, bag_clf_50.predict(X_test))
print(f"Dokładność Bagging 50% dla zbioru treningowego: {bag_acc_train_50}")
print(f"Dokładność Bagging 50% dla zbioru testowego: {bag_acc_test_50}")

Dokładność Bagging 50% dla zbioru treningowego: 0.931924882629108
Dokładność Bagging 50% dla zbioru testowego: 0.6573426573426573


#### Pasting

In [ ]:
pas_clf = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=30,
    bootstrap=False,  # wybór intsancji bez powtórzeń
    random_state=42
)
pas_clf.fit(X_train, y_train)

pas_acc_train = accuracy_score(y_train, pas_clf.predict(X_train))
pas_acc_test = accuracy_score(y_test, pas_clf.predict(X_test))
print(f"Dokładność Pasting dla zbioru treningowego: {pas_acc_train}")
print(f"Dokładność Pasting dla zbioru testowego: {pas_acc_test}")

Dokładność Pasting dla zbioru treningowego: 1.0
Dokładność Pasting dla zbioru testowego: 0.6293706293706294


#### Pasting z wykorzystaniem 50% instancji

In [ ]:
pas_clf_50 = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=30,
    bootstrap=False,  # wybór intsancji bez powtórzeń
    random_state=42,
    max_samples= 0.5  # maksymalnie 50% instancji do trenowania modelu
)
pas_clf_50.fit(X_train, y_train)

pas_acc_train_50 = accuracy_score(y_train, pas_clf_50.predict(X_train))
pas_acc_test_50 = accuracy_score(y_test, pas_clf_50.predict(X_test))
print(f"Dokładność Pasting 50% dla zbioru treningowego: {pas_acc_train_50}")
print(f"Dokładność Pasting 50% dla zbioru testowego: {pas_acc_test_50}")

Dokładność Pasting 50% dla zbioru treningowego: 0.9765258215962441
Dokładność Pasting 50% dla zbioru testowego: 0.6783216783216783


#### Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=30, random_state=42)
rfc.fit(X_train, y_train)

rfc_acc_train = accuracy_score(y_train, rfc.predict(X_train))
rfc_acc_test = accuracy_score(y_test, rfc.predict(X_test))
print(f"Dokładność Random Forest dla zbioru treningowego: {rfc_acc_train}")
print(f"Dokładność Random Forest dla zbioru testowego: {rfc_acc_test}")

Dokładność Random Forest dla zbioru treningowego: 1.0
Dokładność Random Forest dla zbioru testowego: 0.6853146853146853


#### AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ada_clf = AdaBoostClassifier(n_estimators=30, random_state=42)
ada_clf.fit(X_train, y_train)

ada_acc_train = accuracy_score(y_train, ada_clf.predict(X_train))
ada_acc_test = accuracy_score(y_test, ada_clf.predict(X_test))
print(f"Dokładność AdaBoost dla zbioru treningowego: {ada_acc_train}")
print(f"Dokładność AdaBoost dla zbioru testowego: {ada_acc_test}")

Dokładność AdaBoost dla zbioru treningowego: 0.7793427230046949
Dokładność AdaBoost dla zbioru testowego: 0.7062937062937062


#### Gradient Boosting


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb_clf = GradientBoostingClassifier(n_estimators=30, random_state=42)
gb_clf.fit(X_train, y_train)

gb_acc_train = accuracy_score(y_train, gb_clf.predict(X_train))
gb_acc_test = accuracy_score(y_test, gb_clf.predict(X_test))
print(f"Dokładność Gradient Boosting dla zbioru treningowego: {gb_acc_train}")
print(f"Dokładność Gradient Boosting dla zbioru testowego: {gb_acc_test}")

Dokładność Gradient Boosting dla zbioru treningowego: 0.8309859154929577
Dokładność Gradient Boosting dla zbioru testowego: 0.7062937062937062


In [ ]:
# Zapisanie wyników do listy
acc_list2 = [
    (bag_acc_train, bag_acc_test),
    (bag_acc_train_50, bag_acc_test_50),
    (pas_acc_train, pas_acc_test),
    (pas_acc_train_50, pas_acc_test_50),
    (rfc_acc_train, rfc_acc_test),
    (ada_acc_train, ada_acc_test),
    (gb_acc_train, gb_acc_test)
]
acc_list2
with open("acc_bag.pkl", "wb") as file:
  pickle.dump(acc_list2, file)

In [ ]:
# Zapisanie klasyfikatorów do lsit
clf_list2 = [bag_clf, bag_clf_50, pas_clf, pas_clf_50, rfc, ada_clf, gb_clf]
with open("bag.pkl", "wb") as file:
  pickle.dump(clf_list2, file)

### Sampling 2 cech ze wszystkich dostępnych bez powtórzeń z wykorzystaniem 30 drzew decyzyjnych

In [ ]:
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(data_breast_cancer.data, data_breast_cancer.target, test_size=0.2, random_state=42)

bag_sam = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=30,
    max_features=2, # 2 cechy dla każdego drzewa
    bootstrap=True,  # instancje wybierane z powtórzeniami
    max_samples = 0.5,  # każde drzewo dostaje połowę instancji
    bootstrap_features=False # Cechy są wybierane bez powtórzeń
)
bag_sam.fit(X_train_2, y_train_2)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_features=2,
                  max_samples=0.5, n_estimators=30)

In [ ]:
acc_list_2 = [accuracy_score(y_train_2, bag_sam.predict(X_train_2)), accuracy_score(y_test_2, bag_sam.predict(X_test_2))]
acc_list_2
with open("acc_fea.pkl", "wb") as file:
  pickle.dump(acc_list_2, file)

bag_list = [bag_sam]
with open("fea.pkl", "wb") as file:
  pickle.dump(bag_list, file)

#### Sprawdzenie, które cechy dają najlepszą dokładność

In [ ]:
import pandas as pd
df = pd.DataFrame(columns=['train_accuracy', 'test_accuracy', 'features'])

for i, estimator in enumerate(bag_sam.estimators_):
  feature_i = bag_sam.estimators_features_[i]  # indeksy cech
  feature_names = data_breast_cancer.data.columns[feature_i] # nazwy cech

  train_acc = accuracy_score(y_train_2, estimator.predict(X_train_2[feature_names].values))   # values żeby nie było warningu że to DataFrame
  test_acc = accuracy_score(y_test_2, estimator.predict(X_test_2[feature_names].values))

  df.loc[len(df)] = [train_acc, test_acc, feature_names.values]
df.sort_values(by=['test_accuracy', 'train_accuracy'], inplace=True, ascending=False)
df

,train_accuracy,test_accuracy,features
16,0.938462,0.929825,"[mean radius, worst perimeter]"
17,0.942857,0.921053,"[worst radius, worst compactness]"
3,0.931868,0.921053,"[radius error, worst perimeter]"
10,0.929670,0.921053,"[worst fractal dimension, mean perimeter]"
19,0.931868,0.912281,"[mean concavity, area error]"
27,0.936264,0.903509,"[worst texture, mean concave points]"
6,0.883516,0.903509,"[area error, concavity error]"
11,0.927473,0.894737,"[mean concavity, mean perimeter]"
0,0.918681,0.894737,"[area error, worst radius]"
28,0.956044,0.885965,"[worst concave points, mean area]"


In [ ]:
with open("acc_fea_rank.pkl", "wb") as file:
  pickle.dump(df, file)